In [1]:
!pip install javalang

In [2]:
import os
import python_debugger as pyd
import javalang
import xml.etree.ElementTree as ET
import numpy as np

In [3]:
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [4]:
def augmentCode(sourcefileX, changeVariable,permuStatement, task=1 ):
    data=pyd.readJavaFile(sourcefileX)
    tree = javalang.parse.parse(data)
    newCode,_, _ = pyd.AST(tree).getAugmentedCode(task = task, changeVariable=changeVariable, permuStatement=permuStatement)
    return newCode

In [5]:
def inputGenerate(list_newCode,copyNum,txtfiledir,case):
    for i in range(copyNum):
        copyfiledir = txtfiledir + "\\" + str(i)  #create txt case/0 dir, e.g. XX/CASE/0
        if not os.path.exists(copyfiledir):
            os.mkdir(copyfiledir)
            f_input=open(case,'w') #寫檔
            f_input.write(" <BOC> ")
            for newCode in list_newCode:
                ct = newCode.split("\n"); #以換行符號做切割
                print(type(ct))
                for content in ct:
                    print(content)
                    f_input.write(content)
                    f_input.write(" <CR> ")
                f_input.write(" <EOC> ")
            f_input.close()

In [6]:
#permuatation 
#Note alg:
'''
    if i == 0:
        line.append(per[i])
    elif (per[i] == per[i-1]+1 and i != per.end()):
        line.append(per[i])
    elif (per[i] != per[i-1]+1 and i != per.end()):
        lines.append(line)
        line.clear()
        line.append(per[i])
    else:
        lines.append(per[i])
'''
#case 1
#permutation = {1:3, 2:4, 3:9, 4:5, 5:10}
#case 2
#permutation={1:8,2:1,3:2,4:11,5:10,6:15,7:6,8:7}
#case 3
#permutation={1:9,2:5,3:11,4:23,5:8,6:30,7:22,8:7,9:15,10:16}
#begin = 1
#end = 10
def changeLineBlock(begin, end, permutation):
    permu_lineblock = org_lineblock = [value for value in range(begin, end+1)]
    #print("org: ", org_lineblock)
    #make permu_lineblock get permutation value
    for i in range(len(org_lineblock)):
        permu_lineblock[i] = permutation[permu_lineblock[i]]
    #print("permu: ", permu_lineblock)
    #print("permu type :", type(permu_lineblock))
    #sort permu
    permu_lineblock.sort()
    #print("sorted permu: ", permu_lineblock)
    #start permu
    line = [] 
    lines = []
    last_permu = (len(permu_lineblock)-1)
    #print("last permu:", permu_lineblock[last_permu])
    #detail in alg note
    for i in range(len(permu_lineblock)):
        if (i == 0):
            line.append(permu_lineblock[i])
            #print("line1: ", line)
            #print("lines1: ", lines)
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        elif (permu_lineblock[i] == (permu_lineblock[i-1]+1) ):
            line.append(permu_lineblock[i])
            #print("line2: ", line)
            #print("lines2: ", lines)
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        elif (permu_lineblock[i] != (permu_lineblock[i-1]+1) ):
            #print("permu3: ", permu_lineblock[i])
            #print("lines3: ", lines)
            lines.append(list(line))
            #print("lines3 appended: ", lines)
            #print("line3: ", line)
            line.clear()
            line.append(permu_lineblock[i])
            #print("line3 append after clear: ", line)
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        else:
            line.append(permu_lineblock[i])
            lines.append(line)
            line.clear()
            #print("lines4: ", lines)
    return lines 


In [2]:
#can run but have issue
'''
para:
src_dict: permutation (type: dict)
copyNum: copy file times
xmlfiledir: save file dir
case: normalization folder case
'''
def xmlGenerate(src_dict, copyNum, xmlfiledir, case, file):
    for i in range(copyNum):
        copyfiledir = xmlfiledir + "\\" + str(i)  #create txt case/0 dir, e.g. XX/CASE/0
        if not os.path.exists(copyfiledir):
            os.mkdir(copyfiledir)
        tree = ET.parse(file + ".xml") #get xml file 
        root = tree.getroot()# find root 
        for Linelist in root.iter("Linelist"):# 尋找 Linelist 節點
            for line in Linelist.iter("Line"):#在Linelist 尋找 Line 節點
                old_begin = line.get("Begin")# 取得節點指定屬性質
                old_end = line.get("End")
                src = line.get("src")
                line_block = changeLineBlock(old_begin, old_end, src_dict[src])
                for index in line_block:
                    element = Element('line',{'src':src},{'End':index[1]},{'Begin':index[0]})
                    Linelist.append(element)
                root.remove(line)
        tree.write(file + ".xml")
            


In [ ]:
#can run but have issue
'''
para:
src_dict: permutation (type: dict)
copyNum: copy file times
xmlfiledir: save file dir
case: normalization folder case
'''
def XmlGenerate(src_dict, copyNum, xmlfiledir, case, file):
    for i in range(copyNum):
        copyfiledir = xmlfiledir + "\\" + str(i)  #create txt case/0 dir, e.g. XX/CASE/0
        if not os.path.exists(copyfiledir):
            os.mkdir(copyfiledir)
            tree = ET.parse(file + ".xml") #get xml file 
            root = tree.getroot()# find root 
            for Linelist in root.iter("Linelist"):# 尋找 Linelist 節點
                for line in Linelist.iter("Line"):#在Linelist 尋找 Line 節點
                    old_begin = line.get("Begin")# 取得節點指定屬性質
                    old_end = line.get("End")
                    src = line.get("src")
                    line_block = changeLineBlock(old_begin, old_end, src_dict[src])
                    for index in line_block:
                        element = Element('line',{'src':src},{'End':index[1]},{'Begin':index[0]})
                        Linelist.append(element)
                    root.remove(line)
            tree.write(file + ".xml")

In [ ]:
def main():
    if __name__ == '__main__' :
    source_dir = "NormalizedCODES" #the source codes
    dest_dir = "AugmentedCODES" #store the augmented files
    txt_dir = "InputTxt"
    xml_dir = "OutputXml"
    cases = listdir_fullpath(source_dir)
    copyNum = 1
    for case in cases:
        list_newCode=[]
        src_dict={}
        sourcefiledir = source_dir + "\\" + case
        destfiledir = dest_dir + "\\" + case
        if not os.path.exists(destfiledir):
            os.mkdir(destfiledir)  #create dest case dir, e.g. XX/CASE
        txtfiledir = txt_dir + "\\" + case
        if not os.path.exists(txtfiledir):
            os.mkdir(txtfiledir)  #create txt case dir, e.g. XX/CASE
        xmlfiledir = xml_dir + "\\" + case
        if not os.path.exists(xmlfiledir):
            os.mkdir(xmlfiledir)  #create xml case dir, e.g. XX/CASE
        sourcefiles = listdir_fullpath(sourcefiledir)

        for sourcefile in sourcefiles: # .java
            for i in range(copyNum):
                print("Augmenting case : ", case, "sourcefile: ",sourcefile, " copy num: ", i, " ......") 
                copyfiledir = destfiledir + "\\" + str(i)  #create dest case/0 dir, e.g. XX/CASE/0
                if not os.path.exists(copyfiledir):
                    os.mkdir(copyfiledir)  #create dest case/0 dir
                    destfile = copyfiledir + "\\" + sourcefile
                    print("destfile:",destfile)
                    newCode,variableMap, permutation = augmentCode(sourcefile, True, True, 1)
                    list_newCode.append(newCode)
                    src_dict["destfile"]=permutation
                    pyd.write_to_file(newCode, destfile)
            inputGenerate(list_newCode,copyNum,txtfiledir,case)

In [8]:

if __name__ == '__main__' :
    source_dir = "NormalizedCODES" #the source codes
    dest_dir = "AugmentedCODES" #store the augmented files
    txt_dir = "InputTxt"
    xml_dir = "OutputXml"
    cases = listdir_fullpath(source_dir)
    copyNum = 1
    for case in cases:
        list_newCode=[]
        src_dict={}
        sourcefiledir = source_dir + "\\" + case
        destfiledir = dest_dir + "\\" + case
        if not os.path.exists(destfiledir):
            os.mkdir(destfiledir)  #create dest case dir, e.g. XX/CASE
        txtfiledir = txt_dir + "\\" + case
        if not os.path.exists(txtfiledir):
            os.mkdir(txtfiledir)  #create txt case dir, e.g. XX/CASE
        xmlfiledir = xml_dir + "\\" + case
        if not os.path.exists(xmlfiledir):
            os.mkdir(xmlfiledir)  #create xml case dir, e.g. XX/CASE
        sourcefiles = listdir_fullpath(sourcefiledir)
        
        for sourcefile in sourcefiles: # .java
            for i in range(copyNum):
                print("Augmenting case : ", case, "sourcefile: ",sourcefile, " copy num: ", i, " ......") 
                copyfiledir = destfiledir + "\\" + str(i)  #create dest case/0 dir, e.g. XX/CASE/0
                if not os.path.exists(copyfiledir):
                    os.mkdir(copyfiledir)  #create dest case/0 dir
                    destfile = copyfiledir + "\\" + sourcefile
                    print("destfile:",destfile)
                    newCode,variableMap, permutation = augmentCode(sourcefile, True, True, 1)
                    list_newCode.append(newCode)
                    src_dict["destfile"]=permutation
                    pyd.write_to_file(newCode, destfile)
                    
        inputGenerate(list_newCode,copyNum,txtfiledir,case)

In [ ]:
if __name__ == '__main__' :
    source_dir = "D:/NewVerPythonDebugger/NormalizedCODES" #the source codes
    xml_source_dir ="D:/NewVerPythonDebugger/SourceXml"
    dest_dir = "D:/NewVerPythonDebugger/AugmentedCODES" #store the augmented files
    txt_dir = "D:/NewVerPythonDebugger/InputTxt"
    xml_dir = "D:/NewVerPythonDebugger/OutputXml"
    cases = listdir_fullpath(source_dir)
    print("cases",cases)
    print("type(cases[0])",type(cases[0]))
    xml_cases = listdir_fullpath(xml_source_dir)
    copyNum = 1
    
    for case in cases:
        list_newCode=[]
        list_src_dict={}
        sourcefiledir = source_dir + "/" + case
        #print("sourcefiledir",sourcefiledir)
        destfiledir = dest_dir + "/" + case
        if not os.path.exists(destfiledir):
            os.mkdir(destfiledir)  #create dest case dir, e.g. XX/CASE
        txtfiledir = txt_dir + "/" + case
        if not os.path.exists(txtfiledir):
            os.mkdir(txtfiledir)  #create txt case dir, e.g. XX/CASE
        xmlfiledir = xml_dir + "/" + case
        if not os.path.exists(xmlfiledir):
            os.mkdir(xmlfiledir)  #create xml case dir, e.g. XX/CASE
        sourcefiles = listdir_fullpath(sourcefiledir)
        for sourcefile in sourcefiles: # .java
            for i in range(copyNum):
                #print("Augmenting case : ", case, "sourcefile: ",sourcefile, " copy num: ", i, " ......") 
                copyfiledir = destfiledir + "/" + str(i)  #create dest case/0 dir, e.g. XX/CASE/0
                if not os.path.exists(copyfiledir):
                    os.mkdir(copyfiledir)  #create dest case/0 dir
                destfile = copyfiledir + "/" + sourcefile
                #print("destfile:",destfile)
                newCode, permutation = augmentCode(sourcefiledir+"/"+sourcefile, True, True, 1)
                pyd.write_to_file(newCode, destfile)
            
        for i in range(copyNum):
            for sourcefile in sourcefiles:
                newCode, permutation = augmentCode(sourcefiledir+"/"+sourcefile, True, True, 1)
                #print("type(sourcefile)",type(sourcefile))
                src=sourcefile.split(".java")
                #print("src[0]",src[0])
                list_newCode.append(newCode)
                #print("list_newCode",list_newCode)
                src_dict={src[0]:permutation}
                list_src_dict.update(src_dict)
                #print("list_src_dict",list_src_dict)
            
            inputGenerate(list_newCode,copyNum,txtfiledir,case)
            #xmlGenerate(src_dict,copyNum,xmlfiledir,case)